# SEIFA Integration with baseline dataset


### Note on SEIFA Data Year


The SEIFA (Socio-Economic Indexes for Areas) data used in this analysis is based on the 2021 Census. While our healthcare cost and PPH datasets are from 2022–23, SEIFA is included as a socio-economic baseline indicator to explore potential associations between socio-economic status and healthcare outcomes.
We acknowledge that socio-economic rankings may have changed slightly between 2021 and 2023; however, SEIFA rankings are generally stable over short periods, and the 2021 values are the most recent available at the time of analysis.
Any findings involving SEIFA should be interpreted with this context in mind.

In [2]:
# importing the required libraries

import pandas as pd
import numpy as np

In [3]:
# Loading the seifa data file

seifa_path = "../Data/raw-data/Statistical_Area_Level 2_Indexes_SEIFA_2021.xlsx"
xls = pd.ExcelFile(seifa_path)

# Check available sheets
xls.sheet_names

# Loading the required datasheet(Table 1) for next steps
df_seifa =  xls.parse('Table 1', skiprows=5, dtype=str)

# Previewing the top rows of the loaded dataframe
df_seifa.head(10)


,2021 Statistical Area Level 2 (SA2) 9-Digit Code,2021 Statistical Area Level 2 (SA2) Name,Score,Decile,Score.1,Decile.1,Score.2,Decile.2,Score.3,Decile.3,Usual Resident Population
0,101021007,Braidwood,1024,6,1001,6,1027,7,1008,6,4343
1,101021008,Karabar,994,5,982,5,1000,5,967,5,8517
2,101021009,Queanbeyan,1010,5,998,6,945,3,1000,6,11342
3,101021010,Queanbeyan - East,1025,6,1015,6,969,4,1025,7,5085
4,101021012,Queanbeyan West - Jerrabomberra,1098,10,1107,9,1109,10,1080,8,12744
5,101021610,Googong,1128,10,1154,10,1165,10,1127,9,6224
6,101021611,Queanbeyan Surrounds,1098,10,1105,9,1135,10,1073,8,16539
7,101031013,Bombala,965,3,935,3,976,4,919,3,2454
8,101031014,Cooma,982,4,953,4,964,3,943,4,6761
9,101031015,Cooma Surrounds,1039,7,1007,6,1066,9,985,5,3472


### Data filtering

In [4]:
# Renaming and extracting the important columns
df_seifa = df_seifa.rename(columns={
    df_seifa.columns[0]: 'SA2_CODE',
    'Score': 'IRSD_score',
    'Decile': 'IRSD_decile',
    'Usual Resident Population': 'Usual_Resident_Population'
}).loc[:, ['SA2_CODE','IRSD_score','IRSD_decile','Usual_Resident_Population']]

df_filtered_seifa = df_seifa[['SA2_CODE', 'IRSD_score', 'IRSD_decile', 'Usual_Resident_Population']]

# Clean the data types
df_filtered_seifa['SA2_CODE'] = df_filtered_seifa['SA2_CODE'].str.strip()
df_filtered_seifa['IRSD_score'] = pd.to_numeric(df_filtered_seifa['IRSD_score'], errors='coerce')
df_filtered_seifa['IRSD_decile'] = pd.to_numeric(df_filtered_seifa['IRSD_decile'], errors='coerce')
df_filtered_seifa['Usual_Resident_Population'] = pd.to_numeric(df_filtered_seifa['Usual_Resident_Population'], errors='coerce')


df_filtered_seifa.head()



,SA2_CODE,IRSD_score,IRSD_decile,Usual_Resident_Population
0,101021007,1024.0,6.0,4343.0
1,101021008,994.0,5.0,8517.0
2,101021009,1010.0,5.0,11342.0
3,101021010,1025.0,6.0,5085.0
4,101021012,1098.0,10.0,12744.0


In [5]:
# Loading the SA2 -> PHN Mapping file

phn_sa2_map_path = "../Data/raw-data/primary-health-networks-phn-2023-statistical-area-level-2-2021.xlsx"
phn_sa2_map_xls = pd.ExcelFile(phn_sa2_map_path)

# Check available sheets
phn_sa2_map_xls.sheet_names

# Loading the required datasheet(Table 1) for next steps
df_phn_sa2_map =  phn_sa2_map_xls.parse('CG_SA2_2021_PHN_2017_All', dtype=str)

# Previewing the top rows of the loaded dataframe
df_phn_sa2_map.head(10)

,SA2_CODE_2021,SA2_NAME_2021,PHN_CODE_2023,PHN_NAME_2023,RATIO_FROM_TO,INDIV_TO_REGION_QLTY_INDICATOR,OVERALL_QUALITY_INDICATOR,BMOS_NULL_FLAG,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18
0,101021007,Braidwood,PHN106,South Eastern NSW,1,Good,Good,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,101021008,Karabar,PHN106,South Eastern NSW,1,Good,Good,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,101021009,Queanbeyan,PHN106,South Eastern NSW,1,Good,Good,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,101021010,Queanbeyan - East,PHN106,South Eastern NSW,1,Good,Good,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,101021012,Queanbeyan West - Jerrabomberra,PHN106,South Eastern NSW,1,Good,Good,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,101021610,Googong,PHN106,South Eastern NSW,1,Good,Good,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,101021611,Queanbeyan Surrounds,PHN106,South Eastern NSW,1,Good,Good,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,101031013,Bombala,PHN106,South Eastern NSW,1,Good,Good,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,101031014,Cooma,PHN106,South Eastern NSW,1,Good,Good,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,101031015,Cooma Surrounds,PHN106,South Eastern NSW,1,Good,Good,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Data filtering

In [6]:
# Renaming and extracting the important columns
df_phn_sa2_map = df_phn_sa2_map.rename(columns={
   	'SA2_CODE_2021': 'SA2_CODE',
    'PHN_CODE_2023': 'PHN_Code',
    'PHN_NAME_2023': 'PHN_Name',
    'RATIO_FROM_TO': 'Ratio'
})

df_phn_sa2_map = df_phn_sa2_map[['SA2_CODE', 'PHN_Code', 'PHN_Name', 'Ratio']]

# Clean the data types
df_phn_sa2_map['SA2_CODE'] = df_phn_sa2_map['SA2_CODE'].str.strip()
df_phn_sa2_map['PHN_Code'] = df_phn_sa2_map['PHN_Code'].str.strip()
df_phn_sa2_map['PHN_Name'] = df_phn_sa2_map['PHN_Name'].str.strip()
df_phn_sa2_map['Ratio'] = pd.to_numeric(df_phn_sa2_map['Ratio'], errors='coerce')

df_phn_sa2_map.head()

,SA2_CODE,PHN_Code,PHN_Name,Ratio
0,101021007,PHN106,South Eastern NSW,1.0
1,101021008,PHN106,South Eastern NSW,1.0
2,101021009,PHN106,South Eastern NSW,1.0
3,101021010,PHN106,South Eastern NSW,1.0
4,101021012,PHN106,South Eastern NSW,1.0


In [7]:
# Merging SEIFA data with mapping data
df_seifa_phn = df_phn_sa2_map.merge(df_filtered_seifa, on='SA2_CODE', how='left')

df_seifa_phn.head(10)

,SA2_CODE,PHN_Code,PHN_Name,Ratio,IRSD_score,IRSD_decile,Usual_Resident_Population
0,101021007,PHN106,South Eastern NSW,1.0,1024.0,6.0,4343.0
1,101021008,PHN106,South Eastern NSW,1.0,994.0,5.0,8517.0
2,101021009,PHN106,South Eastern NSW,1.0,1010.0,5.0,11342.0
3,101021010,PHN106,South Eastern NSW,1.0,1025.0,6.0,5085.0
4,101021012,PHN106,South Eastern NSW,1.0,1098.0,10.0,12744.0
5,101021610,PHN106,South Eastern NSW,1.0,1128.0,10.0,6224.0
6,101021611,PHN106,South Eastern NSW,1.0,1098.0,10.0,16539.0
7,101031013,PHN106,South Eastern NSW,1.0,965.0,3.0,2454.0
8,101031014,PHN106,South Eastern NSW,1.0,982.0,4.0,6761.0
9,101031015,PHN106,South Eastern NSW,1.0,1039.0,7.0,3472.0


In [8]:
# Adjusting seifa based on the population and ratio

# Adjusting population with the ratio
df_seifa_phn['Weighted_pop'] = df_seifa_phn['Usual_Resident_Population'] * df_seifa_phn['Ratio']

# Calculating weighted score
df_seifa_phn['Weighted_score'] = df_seifa_phn['IRSD_score'] * df_seifa_phn['Weighted_pop']

print("Rows lacking IRSD_score:", df_seifa_phn['IRSD_score'].isna().sum())
df_seifa_phn.head()

Rows lacking IRSD_score: 127


,SA2_CODE,PHN_Code,PHN_Name,Ratio,IRSD_score,IRSD_decile,Usual_Resident_Population,Weighted_pop,Weighted_score
0,101021007,PHN106,South Eastern NSW,1.0,1024.0,6.0,4343.0,4343.0,4447232.0
1,101021008,PHN106,South Eastern NSW,1.0,994.0,5.0,8517.0,8517.0,8465898.0
2,101021009,PHN106,South Eastern NSW,1.0,1010.0,5.0,11342.0,11342.0,11455420.0
3,101021010,PHN106,South Eastern NSW,1.0,1025.0,6.0,5085.0,5085.0,5212125.0
4,101021012,PHN106,South Eastern NSW,1.0,1098.0,10.0,12744.0,12744.0,13992912.0


In [9]:
# Aggregating to phn level

seifa_phn_agg = (
    df_seifa_phn
    .groupby(['PHN_Code','PHN_Name'], as_index=False)
    .agg(
        PHN_Pop = ('Weighted_pop','sum'),
        IRSD_Weighted_Score_sum = ('Weighted_score','sum'),
        IRSD_Decile_Mean = ('IRSD_decile','mean')
    )
)

seifa_phn_agg['PHN_Population'] = seifa_phn_agg['PHN_Pop'].round(0).astype(int)
seifa_phn_agg['SEIFA_IRSD_Score'] = (seifa_phn_agg['IRSD_Weighted_Score_sum'] / seifa_phn_agg['PHN_Pop']).round(3)
seifa_phn_agg['IRSD_Decile_Mean'] = seifa_phn_agg['IRSD_Decile_Mean'].round(3)

seifa_phn_agg = seifa_phn_agg[['PHN_Code','PHN_Name','SEIFA_IRSD_Score','IRSD_Decile_Mean','PHN_Population']]

# print(seifa_phn_agg.describe(include='all'))

seifa_phn_agg.head()

,PHN_Code,PHN_Name,SEIFA_IRSD_Score,IRSD_Decile_Mean,PHN_Population
0,PHN101,Central and Eastern Sydney,1033.035,6.871,1540187
1,PHN102,Northern Sydney,1085.809,9.188,922267
2,PHN103,Western Sydney,997.145,5.857,1052966
3,PHN104,Nepean Blue Mountains,1005.928,5.719,383824
4,PHN105,South Western Sydney,931.202,4.225,1059102


In [ ]:
# Saving  PHN-level SEIFA table 
seifa_phn_agg.to_csv('../Data/preprocessed-data/seifa_phn.csv', index=False)

: 